In [2]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("test") \
    .config("spark.jars", "delta-core_2.12-0.7.0.jar") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

spark.sparkContext.addPyFile("delta-core_2.12-0.7.0.jar")
from delta.tables import *

In [7]:
from IPython.display import display

In [8]:
spark.read.parquet(r'C:\work\delta\fullLoad.parquet').show()

+----------+--------------------+----------+-------------------+
|customerId|             address|      city|          createdOn|
+----------+--------------------+----------+-------------------+
|         3|current address f...|chapel hil|2020-01-01 04:26:15|
|         1|current address f...|    durham|2020-01-02 02:27:34|
|         2|current address f...|      cary|2020-01-01 07:34:53|
|         1|   old address for 1|    durham|2020-01-01 02:34:54|
+----------+--------------------+----------+-------------------+



None

In [15]:
path = "/data/DemoData/"
DeltaTable.forPath(spark, path).toDF().drop('createdOn').show()

+----------+--------------------+----------+--------------------+--------------------+-------------------+-------------------+
|customerId|             address|      city|             hashKey|            hashData|          startDate|            endDate|
+----------+--------------------+----------+--------------------+--------------------+-------------------+-------------------+
|         3|current address f...|chapel hil|4e07408562bedb8b6...|0e6be27212bd4e28e...|2020-01-01 04:26:15|               null|
|         2|current address f...|      cary|d4735e3a265e16eee...|fa1e24b2a865f8cae...|2020-01-01 07:34:53|               null|
|         1|   old address for 1|    durham|6b86b273ff34fce19...|1b8c9703b187ae587...|2020-01-01 02:34:54|2020-01-02 02:27:34|
|         1|current address f...|    durham|6b86b273ff34fce19...|46dbc1691eecc26b6...|2020-01-02 02:27:34|               null|
+----------+--------------------+----------+--------------------+--------------------+-------------------+-----

In [12]:
spark.read.parquet(r'C:\work\delta\incLoad.parquet').show()

+----------+--------------------+-------+-------------------+
|customerId|             address|   city|          createdOn|
+----------+--------------------+-------+-------------------+
|         3|current address f...|   apex|2018-04-04 09:32:23|
|         4|   new address for 4|Raleigh|2018-04-04 04:49:22|
|         1|   new address for 1| durham|2018-03-03 02:34:65|
+----------+--------------------+-------+-------------------+



In [16]:
DeltaTable.forPath(spark, path).toDF().drop('createdOn').show()

+----------+--------------------+----------+--------------------+--------------------+-------------------+-------------------+
|customerId|             address|      city|             hashKey|            hashData|          startDate|            endDate|
+----------+--------------------+----------+--------------------+--------------------+-------------------+-------------------+
|         3|current address f...|chapel hil|4e07408562bedb8b6...|0e6be27212bd4e28e...|2020-01-01 04:26:15|2018-04-04 09:32:23|
|         2|current address f...|      cary|d4735e3a265e16eee...|fa1e24b2a865f8cae...|2020-01-01 07:34:53|               null|
|         4|   new address for 4|   Raleigh|4b227777d4dd1fc61...|8662a510123fb5c66...|2018-04-04 04:49:22|               null|
|         1|current address f...|    durham|6b86b273ff34fce19...|46dbc1691eecc26b6...|2020-01-02 02:27:34|2018-03-03 02:34:65|
|         1|   old address for 1|    durham|6b86b273ff34fce19...|1b8c9703b187ae587...|2020-01-01 02:34:54|2020-

In [20]:
from pyspark.sql.functions import *
DeltaTable.forPath(spark, path).toDF().drop('createdOn').filter(col('endDate').isNull()).show()

+----------+--------------------+-------+--------------------+--------------------+-------------------+-------+
|customerId|             address|   city|             hashKey|            hashData|          startDate|endDate|
+----------+--------------------+-------+--------------------+--------------------+-------------------+-------+
|         3|current address f...|   apex|4e07408562bedb8b6...|a25d8362438ee44e0...|2018-04-04 09:32:23|   null|
|         2|current address f...|   cary|d4735e3a265e16eee...|fa1e24b2a865f8cae...|2020-01-01 07:34:53|   null|
|         4|   new address for 4|Raleigh|4b227777d4dd1fc61...|8662a510123fb5c66...|2018-04-04 04:49:22|   null|
|         1|   new address for 1| durham|6b86b273ff34fce19...|99b862ad30cac4bec...|2018-03-03 02:34:65|   null|
+----------+--------------------+-------+--------------------+--------------------+-------------------+-------+

